# Testing Pipelines

## Preprocessing pipeline

### Visualizing workflows

In [3]:
# %pip install llama-index-utils-workflow

import os
from llama_index.utils.workflow import draw_all_possible_flows

from evidence_seeker import (
    PreprocessingSeparateListingsWorkflow,
    SimplePreprocessingWorkflow
)

# create dir ../tmp if not exists
os.makedirs("../tmp", exist_ok=True)

draw_all_possible_flows(
    PreprocessingSeparateListingsWorkflow, filename="../tmp/PreprocessingSeparateListingsWorkflow.html"
)

draw_all_possible_flows(
    SimplePreprocessingWorkflow, filename="../tmp/SimplePreprocessingWorkflow.html"
)

<class 'NoneType'>
<class 'evidence_seeker.preprocessing.ListAscriptiveClaimsBasedOnAscriptiveAnalysisEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'evidence_seeker.preprocessing.ListDescriptiveClaimsBasedOnDescriptiveAnalysisEvent'>
<class 'evidence_seeker.preprocessing.AscriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.DescriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.NormativeAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.CollectCheckedClaimsEvent'>
<class 'evidence_seeker.preprocessing.ListNormativeClaimsBasedOnNormativeAnalysisEvent'>
<class 'evidence_seeker.preprocessing.NormativeAnalysisEvent'>
<class 'evidence_seeker.preprocessing.DescriptiveAnalysisEvent'>
<class 'evidence_seeker.preprocessing.AscriptiveAnalysisEvent'>
<class 'evidence_seeker.preprocessing.NegateClaimEvent'>
../tmp/PreprocessingSeparateListingsWorkflow.html
<class 'NoneType'>
<class 'evidence_seeker.preprocessing.ListClaimsEvent'>
<class 'llama_

### Initiating observation via Phoenix

In [1]:
# %pip pip install arize-phoenix
# %pip install llama-index-callbacks-arize-phoenix
# observability
import phoenix as px
px.launch_app()

import llama_index.core
llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


### Running Workflows directly

In [2]:

import yaml
from pprint import pprint

from evidence_seeker import (
    PreprocessingSeparateListingsWorkflow,
    SimplePreprocessingWorkflow,
    get_openai_llm
)


config_version = "v0.1"
config_file = "../config.yaml" 

with open(config_file, 'r') as file:
    config_dict = yaml.safe_load(file)
    if config_dict['config_version'] != config_version:
        raise ValueError(f"The version of the config file does not match the used version ({config_version})")
    
#pprint(config_dict)

api_key_name = config_dict['models'][config_dict['used_model']]['api_key_name']
model = config_dict['models'][config_dict['used_model']]['model']
base_url = config_dict['models'][config_dict['used_model']]['base_url']
llm=get_openai_llm(api_key_name=api_key_name, model=model, base_url=base_url, max_tokens=2048)

# pw = PreprocessingSeparateListingsWorkflow(timeout=120, verbose=False, llm=llm, config=config_dict)
pw = SimplePreprocessingWorkflow(timeout=120, verbose=False, llm=llm, config=config_dict)

result = await pw.run(claim="Climate change is the biggest challenge for human kind.")
pprint(result)

Used api key name: kideku_toxicity_app_nim
Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://huggingface.co/api/integrations/dgx/v1).
Analysing ascriptive aspects of claim.
Analysing descriptive aspects of claim.
Analysing normative aspects of claim.
Number of claims: 11
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
{'checked_claims': [CheckedClaim(text='Climate change poses a significant threat to human well-being or survival', negation='Climate change does not pose a significant threat to human well-being or survival.', uid='7df4e602-0aee-488f-a6c9-6c878d540ae1', n_evidence=None, average_confirmation=None, evidential_uncertainty=None, verbalized_confirmation=None, documents=None, confirmation_by_document=None, metadata={}),
                    CheckedClaim(text='Climate change is likely to compromise human food secu

### Running ClaimPreprocessor

In [1]:
import yaml
from pprint import pprint

from evidence_seeker import ClaimPreprocessor

config_version = "v0.1"
config_file = "../config.yaml" 

with open(config_file, 'r') as file:
    config_dict = yaml.safe_load(file)
    if config_dict['config_version'] != config_version:
        raise ValueError(f"The version of the config file does not match the used version ({config_version})")

preprocessor = ClaimPreprocessor(config=config_dict)
checked_claims = await preprocessor(claim="Climate change is the biggest challenge for human kind.") 


Init open ai model meta-llama/Llama-3.1-70B-Instruct (from https://huggingface.co/api/integrations/dgx/v1) with temperature=0.2, 
                max_tokens=4096 and context_window=32000
Used api key name: kideku_toxicity_app_nim
Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct, base_url: https://huggingface.co/api/integrations/dgx/v1).
Analysing ascriptive aspects of claim.
Analysing descriptive aspects of claim.
Analysing normative aspects of claim.
Number of claims: 7
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.
Negating claim.


In [2]:
pprint(checked_claims)

{'ascriptive_claims': [CheckedClaim(text='According to the IPCC, climate change is the most significant threat to human societies and ecosystems in the 21st century', negation='According to the IPCC, climate change is not the most significant threat to human societies and ecosystems in the 21st century.', uid='84ce184f-5599-4843-9107-8996191817df', n_evidence=None, average_confirmation=None, evidential_uncertainty=None, verbalized_confirmation=None, documents=None, confirmation_by_document=None, metadata={}),
                       CheckedClaim(text='Climate change is widely considered the biggest challenge facing human kind in terms of its potential impact on global food security, economic stability, and human health', negation='Climate change is not widely considered the biggest challenge facing human kind in terms of its potential impact on global food security, economic stability, and human health.', uid='3fcc3559-09d0-45e0-a43b-4202cca8fa15', n_evidence=None, average_confirmation=